# 🎯 Problema de Negócio Principal:
Como prever o comportamento das ações da VALE3 para apoiar decisões estratégicas de compra e venda? 
Com base em dados históricos de preços e volume, o objetivo é construir um modelo que possa prever o retorno diário(preço da ação para os proximos 7 dias). Isso pode ser usado para:

* Estratégias de trading algorítmico;

* Gestão de risco e identificação de dias anômalos;

* Criação de indicadores preditivos de tendência.

* Importação das bibliotecas necessárias:

In [ ]:
import pandas as pd # type: ignore
import numpy as np # type: ignore
import matplotlib.pyplot as plt # type: ignore
import seaborn as sns # type: ignore
import yfinance as yf # type: ignore
from statsmodels.tsa.statespace.sarimax import SARIMAX # type: ignore
from sklearn.preprocessing import MinMaxScaler # type: ignore
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import LSTM, Dense # type: ignore
from xgboost import XGBRegressor # type: ignore
from sklearn.metrics import mean_absolute_error, mean_squared_error # type: ignore
import warnings
warnings.filterwarnings('ignore')

# 1. Coletando e tratando os dados. 

In [2]:
# Baixando os preços históricos da VALE3:
df_Vale = yf.download(tickers='VALE3.SA', start='2022-01-01', end='2025-08-01', multi_level_index=False)
df_Vale = df_Vale[['Close', 'Open', 'Volume']]
df_Vale.reset_index(inplace=True)
df_Vale['Date'] = pd.to_datetime(df_Vale['Date'])
df_Vale.set_index('Date', inplace=True)
df_Vale.rename(columns={
    'Close': 'Close_VALE3',
    'Open': 'Open_VALE3',
    'Volume': 'Volume_VALE3'
}, inplace=True)

# Baixando os preços históricos do minério de ferro:
df_ferro = pd.read_csv("Dados Históricos - Minério de ferro refinado 62% Fe CFR Futuros.csv")
df_ferro.reset_index(inplace=True)
df_ferro['Data'] = pd.to_datetime(df_ferro['Data'], dayfirst=True)
df_ferro.set_index('Data', inplace=True)
df_ferro = df_ferro[['Último','Abertura', 'Var%']]
# Invertendo a ordem das datas:
df_ferro = df_ferro.sort_index(ascending=True)
df_ferro.rename(columns={
    'Último': 'Close_Ferro',
    'Abertura': 'Open_Ferro',
    'Var%': 'Variacao_ferro'
}, inplace=True) 

# Agora vamos unir os dataframes:

df = pd.merge(df_Vale, df_ferro, left_index=True, right_index=True, suffixes=('_VALE3', '_Ferro'))

# Variação percentual do preço de fechamento da VALE3:
df['Variação_VALE3'] = df_Vale['Close_VALE3'].pct_change() * 100
# Trocando nan por 0%:
df['Variação_VALE3'].fillna(0, inplace=True)

# Vamos transformar as colunas Close_Minerio, Open_Minerio e Variação_Minerio em float:
df['Close_Ferro'] = df['Close_Ferro'].str.replace(',', '.', regex=False).astype(float)
df['Open_Ferro'] = df['Open_Ferro'].str.replace(',', '.', regex=False).astype(float)
df['Variacao_ferro'] = df['Variacao_ferro'].str.replace('%', '', regex=False).str.replace(',', '.', regex=False).astype(float)

# Tratamento para transforma o index em datetime após o merge:
df.reset_index(inplace=True)
df['index'] = pd.to_datetime(df['index'], dayfirst=True)
df.rename(columns={'index': 'Data'}, inplace=True)
df.set_index('Data', inplace=True)

df

[*********************100%***********************]  1 of 1 completed


,Close_VALE3,Open_VALE3,Volume_VALE3,Close_Ferro,Open_Ferro,Variacao_ferro,Variação_VALE3
Data,,,,,,,
2022-01-03,57.766407,58.507002,18557200,120.40,120.40,7.02,0.000000
2022-01-04,57.085064,58.144115,18178700,120.91,120.91,0.42,-1.179480
2022-01-05,57.625710,57.299851,22039000,124.14,124.14,2.67,0.947088
2022-01-06,58.788437,58.240398,22044100,125.94,125.94,1.45,2.017723
2022-01-07,62.209976,59.543835,35213100,126.21,126.21,0.21,5.820089
...,...,...,...,...,...,...,...
2025-07-25,55.700001,56.180000,21918900,98.55,98.55,-0.03,-1.468243
2025-07-28,55.160000,54.939999,19175100,98.67,98.67,0.12,-0.969481
2025-07-29,54.820000,55.549999,14648700,98.98,98.98,0.31,-0.616389


# 2. Adicionando features no nosso DataFrame.

* Médias móveis de 7 dias, 20 dias e 200 dias para suavizar as oscilações de preços e identificar tendências de forma mais clara:

In [3]:
janelas = [7,20,200]

for janela in janelas:
    df[f'MM_{janela}D'] = df['Close_VALE3'].rolling(window=janela, min_periods=1).mean()

df[['MM_7D', 'MM_20D', 'MM_200D']]

,MM_7D,MM_20D,MM_200D
Data,,,
2022-01-03,57.766407,57.766407,57.766407
2022-01-04,57.425735,57.425735,57.425735
2022-01-05,57.492393,57.492393,57.492393
2022-01-06,57.816404,57.816404,57.816404
2022-01-07,58.695119,58.695119,58.695119
...,...,...,...
2025-07-25,56.008571,54.962000,54.664099
2025-07-28,56.131428,55.076500,54.650813
2025-07-29,56.168571,55.185000,54.618465


* Vamos adicionar o retorno diário, e os retornos de sete dias anteriores e 21 dias anteriores:

In [4]:
retornos = [1,7,21]

for retorno in retornos:
    df[f'Retorno_{retorno}D'] = df['Close_VALE3'].pct_change(retorno) * 100
    df[f'Retorno_{retorno}D'].fillna(0, inplace=True)

df[['Retorno_1D', 'Retorno_7D', 'Retorno_21D']]

,Retorno_1D,Retorno_7D,Retorno_21D
Data,,,
2022-01-03,0.000000,0.000000,0.000000
2022-01-04,-1.179480,0.000000,0.000000
2022-01-05,0.947088,0.000000,0.000000
2022-01-06,2.017723,0.000000,0.000000
2022-01-07,5.820089,0.000000,0.000000
...,...,...,...
2025-07-25,-1.468243,2.389704,10.340732
2025-07-28,-0.969481,1.583795,6.076923
2025-07-29,-0.616389,0.476537,3.688294


* Relative Strength Index(14 dias). Um indicador usado para avaliar a força de um movimento
de preço e determinar se um ativo está sobrecarregado ou sobrevendido:

In [6]:
def calcular_RSI(series, windows=14):
    delta = series.diff()

    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    avg_again = gain.rolling(window=windows).mean()
    avg_loss = loss.rolling(window=windows).mean()

    rs = avg_again / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi

df['RSI_14'] = calcular_RSI(df['Close_VALE3'], windows=14)
df['RSI_14'].fillna(df['RSI_14'].mean(), inplace=True)

* O Moving Average Convergence Divergence ajuda a identificar mudanças na força, direção, momento e duração de uma tendência de ações:

In [8]:
MME_12 = df['Close_VALE3'].ewm(span=12, adjust=False).mean()
MME_26 = df['Close_VALE3'].ewm(span=26, adjust=False).mean()

df['MACD'] = MME_12 - MME_26

df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

* As bandas de Bollinger são indicadores usados para medir a volatilidade do mercado e identificar potenciais pontos de compra e venda de ativos: 

In [11]:
# primeiro pegamos as medias móveis de 20 dias:
medias_20D = df['MM_20D']
# E o desvio padrão de 20 dias também:
desvio_20D = df['Close_VALE3'].rolling(window=20, min_periods=1).std()

# Assim podemos calcular as bandas:
df['banda_media'] = medias_20D
df['banda_superior'] = medias_20D + (2 * desvio_20D)
df['banda_inferior'] = medias_20D - (2 * desvio_20D)

# Vamos completar os valores NaN com a média dos dados para cada coluna:
df['banda_inferior'].fillna(df['banda_inferior'].mean(), inplace=True)
df['banda_superior'].fillna(df['banda_superior'].mean(), inplace=True)
df[['banda_media', 'banda_superior', 'banda_inferior']]

,banda_media,banda_superior,banda_inferior
Data,,,
2022-01-03,57.766407,63.262468,55.793055
2022-01-04,57.425735,58.389300,56.462171
2022-01-05,57.492393,58.211802,56.772985
2022-01-06,57.816404,59.239345,56.393464
2022-01-07,58.695119,62.813535,54.576703
...,...,...,...
2025-07-25,54.962000,57.628016,52.295984
2025-07-28,55.076500,57.554268,52.598733
2025-07-29,55.185000,57.390461,52.979539


* O Rolling Sharpe Ratio, ou Razão de Sharpe Móvel, é usado para avaliar o desempenho de um investimento ao longo do tempo, ajustado ao risco:  

In [24]:
def Rolling_Sharpe_Ratio(returns, window=21, risk_free_rate=0.0, trading_days=252):
    # Converter taxa livre de risco para diária:
    rf_daily = (1 + risk_free_rate)**(1/trading_days) - 1

    # Calcular retornos excedentes:
    excess_return = returns - rf_daily

    # Média móvel dos retornos excedentes:
    mean_returns = excess_return.rolling(window).mean()

    # Desvio padrão móvel dos retornos excedentes:
    std_returns = excess_return.rolling(window).std()

    # Sharpe Ratio anualizado:
    sharpe_ratio = mean_returns / std_returns * np.sqrt(trading_days)

    return sharpe_ratio

# Calculando o sharpe ratio de 21 dias( 21 dias = 1 mês de trading):
df['Sharpe_21D'] = Rolling_Sharpe_Ratio(
    returns=df['Retorno_1D'],
    window=21,
    risk_free_rate= 0.15, # 15% ao ano (SELIC atual)
    trading_days=252
)

In [30]:
# Tratando valores NaN do sharpe de 21 dias:
df['Sharpe_21D'].fillna(0, inplace=True)
df['Sharpe_21D'].isnull().sum()

0

* Vamos adicionar a volatilidade de 7 dias e 21 dias:

In [32]:
df['Volatilidade_7D_VALE3'] = np.std(df['Retorno_7D'])
df['Volatilidadde_21D_VALE3'] = np.std(df['Retorno_21D'])
df['Volatilidade_7D_VALE3'].fillna(0, inplace=True)
df['Volatilidadde_21D_VALE3'].fillna(0, inplace=True)

# 3. Normalização, Engenharia de Recursos e Seleção de Recursos: